# 中文分词，词性标注，命名实体识别
使用哈工大本地pyltp进行分词，词性标注，命名实体识别。

In [1]:
from pyltp import Postagger
from pyltp import Segmentor
from pyltp import SentenceSplitter
import jieba.posseg as pseg
import time
import sys
from collections import defaultdict
import collections
import pickle
import json
import os
import time

In [2]:
#分句
def sentenceSplit(sent):
    sents = SentenceSplitter.split(sent)
    return '_'.join(sents)

In [3]:
#词性标注
def get_postags(text_list):
    postags = []
    postagger = Postagger()
    postagger.load('F:/model/3.4/pos.model')
    for text in text_list:
        postag = postagger.postag(text)
        postags.append(list(postag))
    postagger.release()
    return postags

In [4]:
#分词
def segment(filename):
    segmentor = Segmentor()
    segmentor.load('F:/model/3.4/cws.model')
    seg_words = []
    with open(filename,mode='r',encoding='UTF-8') as f:
        for line in f:
            text = line.strip().replace('\n','')
            if len(text) == 0:
                continue
            try:
                text = sentenceSplit(text)
                text = text.split('_')
                cur_words = []
                for t in text:
                    words = segmentor.segment(t)
                    cur_words.extend(list(words))
                seg_words.append(cur_words)
            except:
                print(error)
    segmentor.release()
    return seg_words
            
                

In [5]:
#去除标点符号的分词和词性标注
def pos_word_process(seg_list, pos_list):
    new_seg_list = []
    new_pos_list = []
    posDict = {}
    for words, poses in zip(seg_list,pos_list):
        cur_words = []
        cur_pos = []
        for word, pos in zip(words,poses):
            if pos =='wp':
                continue
            cur_words.append(word)
            cur_pos.append(pos)
            posDict[word] = pos
        new_seg_list.append(cur_words)
        new_pos_list.append(cur_pos)
    return new_seg_list, new_pos_list, posDict

In [6]:
#保存分词和词性标注的结果
def saveList(filename,obj):
    with open(filename, mode='w', encoding='UTF-8') as f:
        for ele in obj:
            if isinstance(ele, collections.Iterable):
                ele = ' '.join(ele)
                f.write(ele+'\n')
            else:
                f.write(str(ele)+'\n')

In [7]:
#命名实体识别，接受分词和词性标注参数
from pyltp import NamedEntityRecognizer
def Ner(word_list, pos_list):
    recognizer = NamedEntityRecognizer()
    recognizer.load('F:/model/3.4/ner.model')
    namedEntity = set()
    n = 1
    for words, poses in zip(word_list, pos_list):
        print('ner',n)
        n += 1
        try:
            if not words:
                continue
            netags = recognizer.recognize(words,poses)
            for id, tags in enumerate(netags):
                if tags != 'O':
                    namedEntity.add(words[id])
            
        except:
            print(error)
    recognizer.release()
    return list(namedEntity)

In [8]:
#保存命名实体识别的结果
def saveSet(filename, obj):
    with open(filename, mode='w',encoding='UTF-8') as f:
        for ele in obj:
            f.write(ele + '\n')

In [9]:
if __name__ == '__main__':
    seg_list = segment('F:/data/comment_clean.txt')
    pos_list = get_postags(seg_list)
    new_seg_list, new_pos_list, posDict = pos_word_process(seg_list,pos_list)
    saveList('F:/data3/comments_clean_seg.txt', new_seg_list)
    saveList('F:/data3/comments_clean_pos.txt',new_pos_list)    

In [10]:
namedEntity = Ner(new_seg_list, new_pos_list)
saveSet('F:/data3/comments_clean_ner.txt', namedEntity)

ner 1
ner 2
ner 3
ner 4
ner 5
ner 6
ner 7
ner 8
ner 9
ner 10
ner 11
ner 12
ner 13
ner 14
ner 15
ner 16
ner 17
ner 18
ner 19
ner 20
ner 21
ner 22
ner 23
ner 24
ner 25
ner 26
ner 27
ner 28
ner 29
ner 30
ner 31
ner 32
ner 33
ner 34
ner 35
ner 36
ner 37
ner 38
ner 39
ner 40
ner 41
ner 42
ner 43
ner 44
ner 45
ner 46
ner 47
ner 48
ner 49
ner 50
ner 51
ner 52
ner 53
ner 54
ner 55
ner 56
ner 57
ner 58
ner 59
ner 60
ner 61
ner 62
ner 63
ner 64
ner 65
ner 66
ner 67
ner 68
ner 69
ner 70
ner 71
ner 72
ner 73
ner 74
ner 75
ner 76
ner 77
ner 78
ner 79
ner 80
ner 81
ner 82
ner 83
ner 84
ner 85
ner 86
ner 87
ner 88
ner 89
ner 90
ner 91
ner 92
ner 93
ner 94
ner 95
ner 96
ner 97
ner 98
ner 99
ner 100
ner 101
ner 102
ner 103
ner 104
ner 105
ner 106
ner 107
ner 108
ner 109
ner 110
ner 111
ner 112
ner 113
ner 114
ner 115
ner 116
ner 117
ner 118
ner 119
ner 120
ner 121
ner 122
ner 123
ner 124
ner 125
ner 126
ner 127
ner 128
ner 129
ner 130
ner 131
ner 132
ner 133
ner 134
ner 135
ner 136
ner 137
ner 138
ner 

In [11]:
#计算词频
def countFrequency(words_list):
    wordFreq = {}
    for words in words_list:
        for word in words:
            if wordFreq.get(word) == None:
                wordFreq[word] = 1
            wordFreq[word] += 1
    return wordFreq

In [12]:
#保存词频，词性标注字典
def saveDict(filename, obj):
    with open(filename, mode='w', encoding='UTF-8') as f:
        for left, right in obj.items():
            f.write(left)
            f.write(':')
            if isinstance(right, collections.Iterable):
                right = ','.join(list(right))
                f.write(right+'\n')
            else:
                f.write(str(right)+'\n')

In [13]:
new_seg_list[0][0] = new_seg_list[0][0].replace('\ufeff','')

In [15]:
wordFreq = countFrequency(new_seg_list)
saveDict('F:/data3/comments_clean_posDict.txt',posDict)
saveDict('F:/data3/comments_clean_wordFreq.txt',wordFreq)
with open('F:/data3/comments_clean_posDict.pkl', 'wb+') as f:
    pickle.dump(posDict, f)
with open('F:/data3/comments_clean_wordFreq.pkl', 'wb+') as f:
    pickle.dump(wordFreq, f)